In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xPX78_8-81ZxK1oCmfCk2AgWsZm69KA1ixpIFpVoUfDt',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'earlydetectionchronickidneydiseas-donotdelete-pr-gacbtbhncnixlb'
object_key = 'Kidney_data.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [2]:
#Importing the packages
import numpy as np
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')  #never print matching warnings
pd.set_option('display.max_columns',26) #Helps to view more colums , kinda widen the display of output 



df.drop ('id', axis= 1, inplace = True)

  #Making the column names meaningful , Renaming !!
df.columns = ['Age','Blood_Pressure','Specific_Gravity','Albumin','Sugar','Red_Blood_Cells','Pus_Cells','Puss_Cell_Clumps','Bacteria',
              'Blood_Gulcose_Random','Blood_Urea','Serum_Creatinine','Sodium','Potassium','Haemoglobin','Packed_Cell_Volume',
              'White_Blood_Cell_Count','Red_Blood_Cell_Count','Hypertension','Diabetes_Mellitus','Coronary_Artery_Disease',
              'Appetite','Peda_Edema','Aanemia', 'clas_s']

df['Red_Blood_Cells'] = df['Red_Blood_Cells'].replace(to_replace = {'normal' : 0, 'abnormal' : 1})
df['Pus_Cells'] = df['Pus_Cells'].replace(to_replace = {'normal' : 0, 'abnormal' : 1})
df['Puss_Cell_Clumps'] = df['Puss_Cell_Clumps'].replace(to_replace = {'notpresent':0,'present':1})
df['Blood_Gulcose_Random'] = df['Blood_Gulcose_Random'].replace(to_replace = {'notpresent':0,'present':1})
df['Hypertension'] = df['Hypertension'].replace(to_replace = {'yes' : 1, 'no' : 0})
df['Diabetes_Mellitus'] = df['Diabetes_Mellitus'].replace(to_replace = {'\tyes':'yes', ' yes':'yes', '\tno':'no'})
df['Diabetes_Mellitus'] = df['Diabetes_Mellitus'].replace(to_replace = {'yes' : 1, 'no' : 0})
df['Coronary_Artery_Disease'] = df['Coronary_Artery_Disease'].replace(to_replace = {'\tno':'no'})
df['Coronary_Artery_Disease'] = df['Coronary_Artery_Disease'].replace(to_replace = {'yes' : 1, 'no' : 0})
df['Appetite'] = df['Appetite'].replace(to_replace={'good':1,'poor':0,'no':np.nan})
df['Peda_Edema'] = df['Peda_Edema'].replace(to_replace = {'yes' : 1, 'no' : 0})
df['Aanemia'] = df['Aanemia'].replace(to_replace = {'yes' : 1, 'no' : 0})
df['clas_s'] = df['clas_s'].replace(to_replace={'ckd\t':'ckd'})
df["clas_s"] = [1 if i == "ckd" else 0 for i in df["clas_s"]]




df.drop ('Bacteria', axis= 1, inplace = True)

df['Packed_Cell_Volume'] = pd.to_numeric(df['Packed_Cell_Volume'], errors='coerce')
df['White_Blood_Cell_Count'] = pd.to_numeric(df['White_Blood_Cell_Count'], errors='coerce')
df['Red_Blood_Cell_Count'] = pd.to_numeric(df['Red_Blood_Cell_Count'], errors='coerce')


features = ['Age','Blood_Pressure','Specific_Gravity','Albumin','Sugar','Red_Blood_Cells','Pus_Cells','Puss_Cell_Clumps',
              'Blood_Gulcose_Random','Blood_Urea','Serum_Creatinine','Sodium','Potassium','Haemoglobin',
              'White_Blood_Cell_Count','Red_Blood_Cell_Count','Hypertension','Diabetes_Mellitus','Coronary_Artery_Disease',
              'Appetite','Peda_Edema','Aanemia']


for feature in features:
    df[feature] = df[feature].fillna(df[feature].median())

#Since all the columns has 2 categories we can go for lable encoder 
#Label Encoding refers to converting the labels into a numeric form so as to convert them into the machine-readable form. 
#Machine learning algorithms can then decide in a better way how those labels must be operated.

from sklearn.preprocessing import LabelEncoder

lab_enc = LabelEncoder()
for i in df:
  df[i] = lab_enc.fit_transform(df[i])


# Independent and Dependent Feature:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X = df[['Haemoglobin', 'Specific_Gravity', 'Red_Blood_Cell_Count', 'Albumin', 'Blood_Urea', 'Blood_Pressure', 'Blood_Gulcose_Random', 'Serum_Creatinine']]

# Train Test Split: 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=33)


# DecisionTreeClassifier:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model = model.fit(X_train,y_train)



## IBM DEPLOYMENT

In [4]:
!pip install -U ibm-watson-machine-learning

     |████████████████████████████████| 1.8 MB 9.5 MB/s eta 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.255
    Uninstalling ibm-watson-machine-learning-1.0.255:
      Successfully uninstalled ibm-watson-machine-learning-1.0.255


In [12]:
from ibm_watson_machine_learning import APIClient
import json

##AUTHENTICATE AND SET SPACE

In [13]:
wml_credentials = {
    "apikey":"9dafixPBcRly9wS1KiSmQf5Ed3irsZnLXW6Jnfuj6MDX",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [14]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------  ------------------------
ID                                    NAME    CREATED
102cd0b2-8c61-4461-96ed-48c691fbf00f  Kidney  2022-11-09T06:35:52.238Z
------------------------------------  ------  ------------------------


In [15]:
 SPACE_ID="102cd0b2-8c61-4461-96ed-48c691fbf00f"

In [16]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [17]:
wml_client.software_specifications.list(500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

##Save and deploy

In [18]:
import sklearn
sklearn.__version__

'1.0.2'

In [19]:
MODEL_NAME = 'kidney'
DEPLOYMENT_NAME = 'kidney'
DEMO_MODEL = model

In [20]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [21]:
# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [22]:
#Save model
model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=X_train, 
    training_target=y_train
)

In [23]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'clas_s',
  'schemas': {'input': [{'fields': [{'name': 'Haemoglobin', 'type': 'int64'},
      {'name': 'Specific_Gravity', 'type': 'int64'},
      {'name': 'Red_Blood_Cell_Count', 'type': 'int64'},
      {'name': 'Albumin', 'type': 'int64'},
      {'name': 'Blood_Urea', 'type': 'int64'},
      {'name': 'Blood_Pressure', 'type': 'int64'},
      {'name': 'Blood_Gulcose_Random', 'type': 'int64'},
      {'name': 'Serum_Creatinine', 'type': 'int64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-09T06:48:35.163Z',
  'id': '87f1c6c0-5380-4308-8e72-c0239f7ec46a',
  'modified_at': '2022-11-09T06:48:39.015Z',
  'name': 'kidney',
  'owner': 'IBMid-667000EZ3P',
  'resource_key': 'f53329fe-8a77-4665-bec6-8fc945ff1f14',
  'space_id': '102cd0b2-8c61-4461

In [24]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

'87f1c6c0-5380-4308-8e72-c0239f7ec46a'

In [25]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [26]:
# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)



#######################################################################################

Synchronous deployment creation for uid: '87f1c6c0-5380-4308-8e72-c0239f7ec46a' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8ad8a3ba-e143-41f6-ac61-01a7f3b24547'
------------------------------------------------------------------------------------------------


